## NYC Airbnb Price Prediction





In [1]:
import numpy as np 
import pandas as pd

# from pycaret.regression import *

from feature_engine.encoding import RareLabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, PowerTransformer, MinMaxScaler
)

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import (
    KFold, RandomizedSearchCV, train_test_split
)

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [3]:
df = pd.read_csv("../Data/airbnb_imputed.csv")

In [4]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,3.420200e+04,3.420200e+04,34202.000000,34202.000000,34202.000000,34202.000000,34202.000000,34202.000000,34202.000000,34202.000000
mean,1.908988e+07,6.776159e+07,40.728840,-73.952123,152.040699,7.058213,23.204930,1.156207,7.180516,113.557862
std,1.098080e+07,7.876830e+07,0.054628,0.046084,234.546984,20.709473,44.327434,1.593658,32.970576,131.911958
min,2.539000e+03,2.438000e+03,40.506410,-74.242850,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.512642e+06,7.858259e+06,40.689920,-73.983020,69.000000,1.000000,1.000000,0.120000,1.000000,0.000000
50%,1.976333e+07,3.087198e+07,40.722900,-73.955680,105.000000,3.000000,5.000000,0.500000,1.000000,46.000000
75%,2.922994e+07,1.074344e+08,40.763148,-73.936123,175.000000,5.000000,23.000000,1.590000,2.000000,230.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.716900,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [ ]:
df.describe(include='object')

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [5]:
df.query("price == 0")
df = df[df['price']>0]

In [6]:
df['last_review'] = pd.to_datetime(df['last_review'])

In [7]:
df['price'] = df['price'].astype("float64")
df['name'] = df['name'].astype("str")
df.dtypes

id                                         int64
name                                      object
host_id                                    int64
host_name                                 object
neighbourhood_group                       object
neighbourhood                             object
latitude                                 float64
longitude                                float64
room_type                                 object
price                                    float64
minimum_nights                             int64
number_of_reviews                          int64
last_review                       datetime64[ns]
reviews_per_month                        float64
calculated_host_listings_count             int64
availability_365                           int64
dtype: object

In [8]:
df['year'] = df.last_review.dt.year
df['month'] = df.last_review.dt.month



In [9]:
X = df.drop(['price', 'host_id', 'host_name', 'id', 'last_review'],axis = 1)
y = df['price']

num_cols = X.select_dtypes(include = np.number).columns.to_list()
cat_cols = X.select_dtypes(exclude = np.number).columns.to_list()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

print(X_train[cat_cols].tail())   

                                                   name neighbourhood_group  \
16855      Gorgeous midtown apartment near Times square           Manhattan   
6267   Your NY home for Quality time , Fully Equipped !            Brooklyn   
11286                       A sunny healthy big bedroom              Queens   
860                                     Bedstuy Bedstay            Brooklyn   
15800               Spacious Flat in HeArt of Bushwick!              Queens   

            neighbourhood        room_type  
16855      Hell's Kitchen  Entire home/apt  
6267             Bushwick  Entire home/apt  
11286            Elmhurst     Private room  
860    Bedford-Stuyvesant  Entire home/apt  
15800           Ridgewood  Entire home/apt  


In [34]:
from sklearn.feature_extraction.text import CountVectorizer
pipe_cat = Pipeline(
    steps = [
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]
)

pipe_num = Pipeline(
    steps = [
    ('scale', MinMaxScaler())
    ]
)

pipe_text = Pipeline(
    steps = [
        #('text_vec', TfidfVectorizer(analyzer='word',
        #                             max_features=25,
        #                             ngram_range=(1,1)))
        ('text_vec', CountVectorizer(max_features=30)
    ]
)


SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' on line 19 (3836359344.py, line 20)

In [12]:
preprocess = ColumnTransformer(
    transformers = [
    ('num', pipe_num, num_cols),
    ('text', pipe_text, 'name'),
    ('cat', pipe_cat, cat_cols)
    ],
    remainder='passthrough'
)
    
preprocess.fit_transform(X_train)

array([[5.80852776e-01, 6.13385303e-01, 2.32185749e-02, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [7.48089300e-01, 5.44690560e-01, 8.00640512e-04, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [5.37510056e-01, 4.89628292e-01, 8.00640512e-04, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [5.92166130e-01, 6.91662706e-01, 8.00640512e-04, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [4.39988938e-01, 6.13556422e-01, 3.20256205e-03, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.02086685e-01, 6.19697690e-01, 8.00640512e-04, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [35]:

model = Pipeline(steps = [
    ('rare', RareLabelEncoder(tol=0.03, variables=['neighbourhood'])),
    ('preprocessor', preprocess),
    ('lgbm', LGBMRegressor())
    #('rf', RandomForestRegressor(n_estimators=800))
])

In [36]:
model_fit = model.fit(X_train, y_train) 
model_fit.score(X_test, y_test)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003362 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7456
[LightGBM] [Info] Number of data points in the train set: 23935, number of used features: 53
[LightGBM] [Info] Start training from score 152.341216


0.1713356082676325

In [37]:
y_pred = model_fit.predict(X_test)

y_pred 

array([ 96.82479139, 157.00026805,  68.78937941, ..., 126.71045571,
        60.20949069,  46.17057124])

In [16]:
print(f"RMSE for LGBM: {np.sqrt(mean_squared_error(y_test, y_pred))},\
\nMAE for LGBM: {mean_absolute_error(y_test, y_pred)}")

RMSE for LGBM: 213.36799639401653,
MAE for LGBM: 65.64066862822675


## Otimização dos parâmetros do modelo

In [26]:
lgbm_grid = {
    'lgbm__num_leaves': [7, 14, 21],
    'lgbm__learning_rate': [0.1, 0.03, 0.001],
    'lgbm__max_depth': [-1, 3, 5],
    'lgbm__n_estimators': [200, 500, 1000],
    'preprocessor__text__text_vec__max_features': [10,20,50]
}

model.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'rare', 'preprocessor', 'lgbm', 'rare__ignore_format', 'rare__max_n_categories', 'rare__missing_values', 'rare__n_categories', 'rare__replace_with', 'rare__tol', 'rare__variables', 'preprocessor__n_jobs', 'preprocessor__remainder', 'preprocessor__sparse_threshold', 'preprocessor__transformer_weights', 'preprocessor__transformers', 'preprocessor__verbose', 'preprocessor__verbose_feature_names_out', 'preprocessor__num', 'preprocessor__text', 'preprocessor__cat', 'preprocessor__num__memory', 'preprocessor__num__steps', 'preprocessor__num__verbose', 'preprocessor__num__scale', 'preprocessor__num__scale__clip', 'preprocessor__num__scale__copy', 'preprocessor__num__scale__feature_range', 'preprocessor__text__memory', 'preprocessor__text__steps', 'preprocessor__text__verbose', 'preprocessor__text__text_vec', 'preprocessor__text__text_vec__analyzer', 'preprocessor__text__text_vec__binary', 'preprocessor__text__text_vec__decode_error', 'preprocessor__tex

In [27]:
lgbm_tune = RandomizedSearchCV(
    model, 
    param_distributions = lgbm_grid,
    cv = 5,
    scoring = 'neg_root_mean_squared_error',
    return_train_score = True,
    n_iter = 5,
    verbose = 1,
    random_state = 42
)

In [28]:
lgbm_tune.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002511 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2244
[LightGBM] [Info] Number of data points in the train set: 19148, number of used features: 38
[LightGBM] [Info] Start training from score 151.810111
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001899 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2264
[LightGBM] [Info] Number of data points in the train set: 19148, number of used features: 38
[LightGBM] [Info] Start training from score 152.350219
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003098 seconds.
You can set `force_r

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('rare',
                                              RareLabelEncoder(tol=0.03,
                                                               variables=['neighbourhood'])),
                                             ('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('num',
                                                                               Pipeline(steps=[('scale',
                                                                                                MinMaxScaler())]),
                                                                               ['latitude',
                                                                                'longitude',
                                                                                'minimum_nights',
                                                                                'number_of_reviews',
                                                                                'reviews_per_month',
                                                                                'calculated_host_listings_count',...
                                                                                'room_type'])])),
                                             ('lgbm', LGBMRegressor())]),
                   n_iter=5,
                   param_distributions={'lgbm__learning_rate': [0.1, 0.03,
                                                                0.001],
                                        'lgbm__max_depth': [-1, 3, 5],
                                        'lgbm__n_estimators': [200, 500, 1000],
                                        'lgbm__num_leaves': [7, 14, 21],
                                        'preprocessor__text__text_vec__max_features': [10,
                                                                                       20,
                                                                                       50]},
                   random_state=42, return_train_score=True,
                   scoring='neg_root_mean_squared_error', verbose=1)

In [32]:
lgbm_tune.best_score_

-215.4803185503108

In [33]:
pd.DataFrame(lgbm_tune.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_preprocessor__text__text_vec__max_features,param_lgbm__num_leaves,param_lgbm__n_estimators,param_lgbm__max_depth,param_lgbm__learning_rate,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,6.475901,1.457594,0.788370,0.084501,10,21,1000,-1,0.1,{'preprocessor__text__text_vec__max_features':...,...,-222.315501,33.212283,5,-105.686541,-101.093169,-99.473031,-108.333114,-106.671552,-104.251481,3.387754
1,4.982218,0.575530,0.734809,0.055096,10,21,200,-1,0.1,{'preprocessor__text__text_vec__max_features':...,...,-215.700337,34.858459,2,-161.035676,-152.754857,-150.232776,-170.034080,-164.174914,-159.646461,7.300015
2,5.777967,0.318140,0.770493,0.026272,50,21,500,5,0.03,{'preprocessor__text__text_vec__max_features':...,...,-215.480319,33.055404,1,-188.564173,-180.292019,-176.789035,-200.132804,-194.792421,-188.114090,8.702802
3,6.063851,0.552459,0.879907,0.064357,20,21,500,5,0.001,{'preprocessor__text__text_vec__max_features':...,...,-221.699453,34.489361,4,-222.514454,-214.271586,-211.191605,-232.289446,-229.635069,-221.980432,8.258081
4,6.727669,0.640267,0.820597,0.074318,20,14,1000,5,0.001,{'preprocessor__text__text_vec__max_features':...,...,-218.083199,34.633252,3,-217.403725,-209.040625,-205.145908,-226.965651,-224.104038,-216.531989,8.399054
